# LLM aids for processing of the first Carlson-Putin interview 

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)   
February 2024

-------

## Introduction

In this notebook we provide aids and computational workflows for the analysis of the first Carlson-Putin interview held in February 9th, 2024.
We mostly use Large Language Models (LLMs). We walk through various steps involved in examining and understanding the interview in a systematic and reproducible manner.

The interview transcripts are taken from [en.kremlin.ru](http://en.kremlin.ru).

The computations are done with a [Raku chatbook](https://raku.land/zef:antononcube/Jupyter::Chatbook), [AAp6, AAv1÷AAv3]. The LLM functions used in the workflows are explained and demonstrated in [AA1, AAv3].
The workflows are done with OpenAI's models [AAp1]; the models of Google (PaLM), [AAp2], and MistralAI, [AAp3], can be also used for the Part 1 summary and the search engine.
The related images were generates with workflows described in [AA2].

### Structure

The structure of the notebook is as follows:

1. **Getting the interview text**   
    Standard ingestion.
2. **Preliminary LLM queries**   
    What are the most important parts or most provocative questions?
3. **Part 1: separation and summary**   
    Overview of the historical review.
4. **Part 2: thematic parts**   
    TLDR via a table of themes.
5. **Interview's spoken parts**    
    Non-LLM extraction of participants' parts.
6. **Search engine**   
    Fast results with LLM embeddings.
7. **Flavored variations**   
    How would Hillary phrase it? And how would Trump answer it?

Sections 5 and 6 can be skipped -- they are (somewhat) more technical.

### Observations

- Using the LLM functions for programmatic access of LLMs speeds up the efforts, I would say, by factor 3-5 times.
- The workflows presented below are fairly universal -- with small changes the notebook can be applied to other interviews.
- Using OpenAI's preview model "gpt-4-turbo-preview" spares or simplifies a fair amount of workflow elements.
    - The model "gpt-4-turbo-preview" takes input with 128K tokens, [OAIb1]. 
    - Hence the whole interview can be processed in one LLM request.
- Since I watched the interview, I can see that the LLM results for most provocative questions or most important statements are good.
    - It is interesting to think about believing those results by people who have not watched the interview.
- The search engine can be replaced or enhanced with a Question Answering System (QAS).
- The flavored variations might be too subtle.
    - I expected more obvious manifestation of characters involved.

-------

## Getting the interview text

The interviews are taken from the dedicated Kremlin's page ["Interview to Tucker Carlson"](http://en.kremlin.ru/events/president/news/73411), hosted at [en.kremlin.ru](http://en.kremlin.ru).

Here we load a package and define a text statistics function:

In [3]:
use HTTP::Tiny;

sub text-stats(Str:D $txt) { <chars words lines> Z=> [$txt.chars, $txt.words.elems, $txt.lines.elems] }

&text-stats

Here we ingest interview's text:

In [4]:
my $url = 'https://raw.githubusercontent.com/antononcube/SimplifiedMachineLearningWorkflows-book/master/Data/Carlson-Putin-interview-2024-02-09-English.txt';
my $txtEN = HTTP::Tiny.new.get($url)<content>.decode;
$txtEN .= subst(/ \v+ /, "\n", :g);
text-stats($txtEN)

(chars => 97354 words => 16980 lines => 292)

------

## Preliminary LLM queries

Here we configure LLM access -- we use OpenAI's model "gpt-4-turbo-preview" since it allows inputs with 128K tokens:

In [185]:
my $conf = llm-configuration('ChatGPT', model => 'gpt-4-turbo-preview', max-tokens => 4096, temperature => 0.2);
$conf.Hash.elems

21

### Questions

First we make an LLM request about the number of questions asked:

In [12]:
llm-synthesize(["How many questions were asked in the following interview?\n\n", $txtEN], e => $conf)

In the interview between Tucker Carlson and President Vladimir Putin, Tucker Carlson asked a total of 57 questions.

Here we ask the questions to be extracted into a JSON list:

In [20]:
my $llmQuestions = llm-synthesize([
    "Extract all questions from the following interview into a JSON list.\n\n", 
    $txtEN,
    llm-prompt('NothingElse')('JSON')
    ], e => $conf, form => sub-parser('JSON'):drop);

deduce-type($llmQuestions)

Vector(Assoc(Atom((Str)), Atom((Str)), 1), 30)

We can see that the LLM-extracted questions are two times less than the LLM-obtained number of questions above.
Here are the extracted questions (in a multi-column format): 

In [25]:
#%html
$llmQuestions>>.values.flat ==> to-html(cols=>3):multicolumn

Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?,In 1654?,What do you think of that?
"You were initially trained in history, as far as I know?",What did he say?,"Evan Gershkovich, that’s a completely different, I mean, this is a thirty-two year old newspaper reporter."
"I beg your pardon, can you tell us what period… I am losing track of where in history we are.",But you haven’t spoken to him since before February of 2022?,He is just a journalist.
"May I ask… You are making the case that Ukraine, certain parts of Ukraine, Eastern Ukraine, in fact, has been Russia for hundreds of years. Why wouldn’t you just take it when you became President 24 years ago? You have nuclear weapons, they don’t. It’s actually your land. Why did you wait so long?",You do not remember?!,"But are you suggesting he was working for the US government or NATO? Or he was just a reporter who was given material he wasn’t supposed to have? Those seem like very different, very different things."
Do you believe Hungary has a right to take back its land from Ukraine? And that other nations have a right to go back to their 1654 borders?,"But he is funding the war that you are fighting, so I think that would be memorable?",Do you think it is too humiliating at this point for NATO to accept Russian control of what was two years ago Ukrainian territory?
Have you told Viktor Orban that he can have a part of Ukraine?,Who blew up Nord Stream?,"Would you be willing to say, “Congratulations, NATO, you won?” And just keep the situation where it is now?"
Why do you think the West rebuffed you then? Why the hostility? Why did the end of the Cold War not fix the relationship? What motivates this from your point of view?,Do you have evidence that NATO or the CIA did it?,Do you think it’s too humiliating at this point for NATO to accept Russian control of what was two years ago Ukrainian territory?
Were you sincere? Would you have joined NATO?,Why wouldn’t you present it and win a propaganda victory?,"Would you be willing to say, “Congratulations, NATO, you won?” And just keep the situation where it is now?"
May I ask what year was this?,Why are they being silent about it? That is very confusing to me. Why wouldn’t the Germans say something about it?,Do you think Zelensky has the freedom to negotiate the settlement to this conflict?
"So, twice you've described US presidents making decisions and then being undercut by their agency heads. So, it sounds like you're describing a system that is not run by the people who are elected, in your telling.",When does the AI empire start do you think?,"Are you worried that what’s happening in Ukraine could lead to something much larger and much more horrible and how motivated are you just to call the US government and say, “let’s come to terms”?"


### Important parts

Here we make a function of extracting *significant* parts from the interview:

In [26]:
my &fProv = llm-function({"Which are the top $^a most $^b in the following interview?\n\n" ~ $txtEN}, e => $conf)

-> **@args, *%args { #`(Block|3154225945208) ... }

#### Most provocative questions

Here we get the most provocative questions:

In [27]:
#% markdown
&fProv(3, 'provocative questions')

Based on the content and context of the interview between Tucker Carlson and President Vladimir Putin, identifying the top three most provocative questions involves subjective judgment. However, considering the potential for controversy, international implications, and the depth of response they elicited, the following three questions can be considered among the most provocative:

1. **NATO Expansion and Perceived Threats to Russia:**
   - **Question:** "On February 24, 2022, you addressed your country in your nationwide address when the conflict in Ukraine started and you said that you were acting because you had come to the conclusion that the United States through NATO might initiate a quote, “surprise attack on our country.” And to American ears that sounds paranoid. Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?"
   - **Context:** This question directly challenges Putin's justification for the military actions in Ukraine, suggesting paranoia, and seeks an explanation for Russia's perceived threat from NATO and the U.S., which is central to understanding the conflict's origins from Russia's perspective.

2. **Possibility of Negotiated Settlement in Ukraine:**
   - **Question:** "Do you think Zelensky has the freedom to negotiate the settlement to this conflict?"
   - **Context:** This question probes the autonomy and authority of Ukrainian President Volodymyr Zelensky in the context of peace negotiations, implicitly questioning the influence of external powers (notably the U.S.) on Ukraine's decision-making and the potential for resolving the conflict through diplomacy.

3. **Use of Nuclear Weapons and Global Conflict:**
   - **Question:** "Do you think NATO was worried about this becoming a global war or nuclear conflict?"
   - **Context:** Given the nuclear capabilities of Russia and the escalating tensions with NATO, this question touches on the fears of a broader, potentially nuclear, conflict. Putin's response could provide insights into Russia's stance on the use of nuclear weapons and its perception of NATO's concerns about escalation.

These questions are provocative due to their direct challenge to Putin's actions and rationale, their exploration of sensitive geopolitical issues, and their potential to elicit responses that could have significant international repercussions.

#### Most important statements

Here we get the most important statements:

In [28]:
#% markdown
&fProv(3, 'important statements')

Based on the extensive interview, the top 3 most important statements that stand out for their significance in understanding the broader context of the conversation and the positions of the involved parties are:

1. **Vladimir Putin's assertion on NATO expansion and its impact on Russia**: Putin's repeated emphasis on NATO's expansion as a direct threat to Russia's security and the broken promises regarding NATO not expanding eastward. This is a critical point as it underlines Russia's longstanding grievance and justification for its actions in Ukraine, reflecting the deep-seated geopolitical tensions between Russia and the West.

2. **Putin's readiness for a negotiated settlement in Ukraine**: Putin's statements indicating a willingness to negotiate a settlement for the conflict in Ukraine, blaming the West and Ukraine for the lack of dialogue and suggesting that the ball is in their court to make amends and come back to the negotiating table. This is significant as it portrays Russia's stance on seeking a diplomatic resolution, albeit under conditions that are likely to favor Russian interests.

3. **Discussion on the potential global implications of the conflict**: The dialogue around the fear of the conflict in Ukraine escalating into a larger, possibly global war, and the mention of nuclear threats. This highlights the high stakes involved not just for the immediate parties but for global security, underscoring the urgency and gravity of finding a peaceful resolution to the conflict.

These statements are pivotal as they encapsulate the core issues at the heart of the Russia-Ukraine conflict, the geopolitical dynamics with NATO and the West, and the potential paths towards resolution or further escalation.

-------

## Part 1: separation and summary

In the first part of the interview Putin gave a historical overview of the formation and evolution of the "Ukrainian lands."
We can extract the first part of the interview "manually" like this:

In [36]:
my ($part1, $part2) = $txtEN.split('Tucker Carlson: Do you believe Hungary has a right to take back its land from Ukraine?');
say "Part 1 stats: {&text-stats($part1)}";
say "Part 2 stats: {&text-stats($part2)}";

Part 1 stats: chars	13629 words	2300 lines	44
Part 2 stats: chars	83639 words	14664 lines	248


Alternatively, we can ask ChatGPT to make the extraction for us:

In [37]:
my $splittingQuestion = llm-synthesize([
    "Which question by Tucker Carlson splits the following interview into two parts:",
    "(1) historical overview Ukraine's formation, and (2) shorter answers.", 
    $txtEN,
    llm-prompt('NothingElse')('the splitting question by Tucker Carlson')
    ], e => $conf)

"May I ask… You are making the case that Ukraine, certain parts of Ukraine, Eastern Ukraine, in fact, has been Russia for hundreds of years. Why wouldn’t you just take it when you became President 24 years ago? You have nuclear weapons, they don’t. It’s actually your land. Why did you wait so long?"

Here is the first part of the interview according to the LLM result:

In [46]:
my $llmPart1 = $txtEN.split($splittingQuestion.substr(10,200)).first;
&text-stats($llmPart1)

(chars => 8751 words => 1486 lines => 29)

**Remark:** We can see that LLM "spared" nearly 1/3 of the "manually" selected text. Below we continue with the latter.

### Summary of the first part

Here is a summary of the first part of the interview:

In [47]:
llm-synthesize(['Summarize the following part one of the Carlson-Putin interview:', $part1], e => $conf);

In the first part of the interview between Tucker Carlson and President Vladimir Putin, Carlson questions Putin about his statement made on February 24, 2022, regarding the conflict in Ukraine. Putin clarifies that he never stated the U.S. would launch a surprise attack on Russia, prompting a discussion on the historical context of Russia-Ukraine relations. Putin traces the origins of the Russian state to 862 and discusses the historical development of the region, including the significance of Kiev and Novgorod, the adoption of Orthodoxy in 988, and the eventual fragmentation and reunification of Russian lands.

Putin explains the historical ties between Russia and Ukraine, mentioning the Polonization efforts in the region and the eventual appeal of the people in parts of what is now Ukraine to Moscow for protection in the 17th century. He provides a detailed historical account of the territorial changes and political maneuvers over the centuries, including the role of the Bolsheviks i

-----

## Part 2: thematic parts

Here we make an LLM request for finding and distilling the themes or the second part of the interview:

In [186]:
my $llmParts = llm-synthesize([
    'Split the following second part of the Tucker-Putin intervew into thematic parts:', 
    $part2,
    "Return the parts as a JSON array.",
    llm-prompt('NothingElse')('JSON')
    ], e => $conf, form => sub-parser('JSON'):drop);

    deduce-type($llmParts)

Vector(Assoc(Atom((Str)), Atom((Str)), 2), 14)

Here we tabulate the found themes:

In [50]:
#%html
$llmParts ==> data-translation(field-names => <theme content>)

theme,content
Historical Borders and Rights of Nations,"Discussion on whether nations have the right to return to their historical borders, referencing the 1654 borders and Stalin's regime."
Hungarian Minority in Ukraine,"Putin shares a personal story from the 1980s about the Hungarian minority in Western Ukraine, emphasizing their historical presence and cultural preservation."
NATO Expansion and Security Concerns,"Putin expresses concerns over NATO expansion and the perceived threat to Russia, mentioning historical agreements and the expectation of cooperation post-Soviet Union collapse."
Relations with Viktor Orban and Ukraine,"Putin denies suggesting to Orban that Hungary could claim parts of Ukraine, while acknowledging the desires of Hungarians in Ukraine to connect with their homeland."
Western Fear of Russia and China,"Discussion on why the West fears a strong Russia and China, with Putin arguing that the West is more afraid of China due to its economic and population size."
NATO's Eastward Expansion,"Putin criticizes NATO's expansion towards Russian borders despite previous assurances, and discusses the impact on Russia's security and relations with the West."
Russian Leadership and Expectations from the West,"Putin reflects on the expectations Russian leadership had from the West post-Soviet Union collapse, including cooperation and non-expansion of NATO."
Ukraine's Political Situation and Relations with Russia,"Putin discusses Ukraine's political changes, the impact of the 2014 coup, and the historical ties between Russia and Ukraine."
Western Support for Ukraine and Sanctions on Russia,"Discussion on the Western support for Ukraine, including military and financial aid, and the impact of sanctions on Russia."
Potential for Negotiations and Peace,"Putin expresses openness to negotiations for peace, criticizing the West and Ukraine for not engaging in dialogue and for the decisions leading to the current conflict."


------

## Interview's spoken parts

In this section we separate the spoken parts of each participant in the interview. We do that using regular expressions, not LLMs.

Here we split the interview text with the names of the participants:

In [6]:
my @parts = $txtEN.split(/ 'Tucker Carlson:' | ['President of Russia' \h+]? 'Vladimir Putin:' /, :v, :skip-empty)>>.Str.rotor(2).map({ Pair.new(|$_) });
say "Total parts {@parts.elems}";

say "First 4:\n";
.say for @parts[^4];


Total parts 149
First 4:

Tucker Carlson: =>  Mr. President, thank you.
On February 24, 2022, you addressed your country in your nationwide address when the conflict in Ukraine started and you said that you were acting because you had come to the conclusion that the United States through NATO might initiate a quote, “surprise attack on our country.” And to American ears that sounds paranoid. Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?

President of Russia Vladimir Putin: =>  It's not that the United States was going to launch a surprise strike on Russia, I didn't say so. Are we having a talk show or a serious conversation?

Tucker Carlson: =>  That was a good quote. Thank you, it’s formidably serious!

Vladimir Putin: =>  You were initially trained in history, as far as I know?



Here we further process the separate participant names and corresponding parts into a list of pairs:

In [7]:
#%html
@parts .= map({ ($_.key.contains('Carlson') ?? 'T.Carlson' !! 'V.Putin') => $_.value.trim });
@parts[^6].Array ==> to-dataset() ==> data-translation

Key,Value
T.Carlson,"Mr. President, thank you. On February 24, 2022, you addressed your country in your nationwide address when the conflict in Ukraine started and you said that you were acting because you had come to the conclusion that the United States through NATO might initiate a quote, “surprise attack on our country.” And to American ears that sounds paranoid. Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?"
V.Putin,"It's not that the United States was going to launch a surprise strike on Russia, I didn't say so. Are we having a talk show or a serious conversation?"
T.Carlson,"That was a good quote. Thank you, it’s formidably serious!"
V.Putin,"You were initially trained in history, as far as I know?"
T.Carlson,Yes.
V.Putin,So if you don’t mind I will take only 30 seconds or one minute of your time to give you a brief historical background.


Here we get all spoken parts of Tucker Carlson (and consider all of them to be "questions"):

In [8]:
my @tcQuestions = @parts.grep({ $_.key.contains('Carlson') && $_.value ~~ / '?' $/ }).map(*.value);
@tcQuestions.elems

53

A table with all questions:

In [9]:
#%html
@tcQuestions ==> to-html(cols=>3):multicolumn

"Mr. President, thank you. On February 24, 2022, you addressed your country in your nationwide address when the conflict in Ukraine started and you said that you were acting because you had come to the conclusion that the United States through NATO might initiate a quote, “surprise attack on our country.” And to American ears that sounds paranoid. Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?",What is denazification? What would that mean?,"Well, let’s just give one example — the US dollar, which has, kind of, united the world in a lot of ways, maybe not to your advantage, but certainly to ours. Is that going away as the reserve currency, the universally accepted currency? How have sanctions, do you think, changed the dollar’s place in the world?"
"May I ask… You are making the case that Ukraine, certain parts of Ukraine, Eastern Ukraine, in fact, has been Russia for hundreds of years. Why wouldn’t you just take it when you became President 24 years ago? You have nuclear weapons, they don’t. It’s actually your land. Why did you wait so long?",Would you be satisfied with the territory that you have now?,"I think that is a fair assessment. The question is what comes next? And maybe you trade one colonial power for another, much less sentimental and forgiving colonial power? Is the BRICS, for example, in danger of being completely dominated by the Chinese economy? In a way that is not good for their sovereignty. Do you worry about that?"
In 1654?,"Really, my question is: What do you do about it? I mean, Hitler has been dead for eighty years, Nazi Germany no longer exists, and it’s true. So, I think, what you are saying, you want to extinguish or at least control Ukrainian nationalism. But how do you do that?","So, you said a moment ago that the world would be a lot better if it were not broken into competing alliances, if there was cooperation globally. One of the reasons you don’t have that is because the current American administration is dead set against you. Do you think if there was a new administration after Joe Biden that you would be able to re-establish communication with the US government? Or does it not matter who the President is?"
"You have, I see, encyclopaedic knowledge of that region. But why didn’t you make this case for the first 22 years as president, that Ukraine wasn’t a real country?","Right. My question is almost specific, it was, of course, not a defense of Nazism. Otherwise, it was a practical question. You don't control the entire country, you don’t seem like you want to. So, how do you eliminate that culture, or an ideology, or feelings, or a view of history, in a country that you don’t control? What do you do about that?","But you are describing two different systems. You say that the leader acts in the interests of the voters, but you also say that these decisions are not made by the leader – they are made by the ruling classes. You have run this country for so long, you have known all these American presidents. What are those power centres in the United States, do you think? And who actually makes the decisions?"
Do you believe Hungary has a right to take back its land from Ukraine? And that other nations have a right to go back to their 1654 borders?,"Well, but you would not be speaking to the Ukrainian president, you would be speaking to the American president. When was the last time you spoke to Joe Biden?",I just have to ask. You have said clearly that NATO expansion eastward is a violation of the promise you were all made in the 1990s. It is a threat to your country. Right before you sent troops into Ukraine the Vice-President of the United States spoke at the Security Conference and encouraged the President of Ukraine to join NATO. Do you think that was an effort to provoke you into military action?
Have you told Viktor Orban that he can have a part of Ukraine?,"But he is funding the war that you are fighting, so I think that would be m

-------

## Search engine

In this section we make a (mini) search engines of the interview parts obtained above.

Here are the steps:

1. Make sure the interview parts are associated with unique identifiers that also identify the speakers
2. Find the embedding vectors for each part.
3. Create a recommendation function that:
   - Filters the embeddings according to specified type 
   - Finds the vector embedding of given query
   - Finds the dot products of query-vector with the part-vectors
   - Pick the top results

Here we make a hash-map of the interview parts obtained above:

In [10]:
my $k = 0;
my %parts = @parts.map({ "{$k++} {$_.key}" => $_.value });
%parts.elems

149

Here we find the LLM embedding vectors of the interview parts:

In [11]:
my %embs = %parts.keys.Array Z=> openai-embeddings(%parts.values, format => 'values').Array;
%embs.elems;

149

Here is a function to find the most relevant parts of the interview for a given query (using dot product):

In [12]:
sub top-parts(Str $query, UInt $n = 3, :$type is copy = 'answers' ) {
    my @vec = |openai-embeddings($query, format=>'values').head;

    if $type.isa(Whatever) { $type = 'part'; }

    my %embsLocal = do given $type {
        when $_ ∈ <part statement> { 
            %embs 
        }
        when $_ ∈ <answer answers Putin> {
            %embs.grep({  $_.key.contains('Putin') })
        }
        when $_ ∈ <question questions Carlson Tucker> {
            %embs.grep({  $_.key.contains('Carlson') })
        }
        default {
            die "Do not know how to process the $type arugment."
        }
    }

    my @sres = %embsLocal.map({ $_.key => sum($_.value >>*<< @vec) });

    @sres .= sort({ - $_.value });
    return @sres[^$n].map({ %(Score => $_.value, Text => %parts{$_.key}) }).Array;
}

&top-parts

Here we find the top 3 results for a query:

In [13]:
#%html
top-parts('Who blew up NordStream 1 and 2?', 3, type => 'part') 
==> data-translation(field-names => <Score Text>)
==> { $_.subst(/:i ('Nord' \h* Stream) /, { '<span style="color: orange">' ~ $0.Str ~ '</span>'}):g }()

Score,Text
0.9778254369990942,Who blew up Nord Stream?
0.8534919207926321,I was busy that day. I did not blow up Nord Stream.
0.8515907005391744,"This also confuses me. But today's German leadership is guided by the interests of the collective West rather than its national interests, otherwise it is difficult to explain the logic of their action or inaction. After all, it is not only about Nord Stream-1, which was blown up, and Nord Stream-2 was damaged, but one pipe is safe and sound, and gas can be supplied to Europe through it, but Germany does not open it. We are ready, please. There is another route through Poland, called Yamal-Europe, which also allows for a large flow. Poland has closed it, but Poland pecks from the German hand, it receives money from pan-European funds, and Germany is the main donor to these pan-European funds. Germany feeds Poland to a certain extent. And they closed the route to Germany. Why? I don't understand. Ukraine, to which the Germans supply weapons and give money. Germany is the second sponsor after the United States in terms of financial aid to Ukraine. There are two gas routes through Ukraine. They simply closed one route, the Ukrainians. Open the second route and get gas from Russia. They do not open it. Why don't the Germans say: “Look, guys, we give you money and weapons. Open up the valve, please, let the gas from Russia pass through for us. We are buying liquefied gas at exorbitant prices in Europe, which brings the level of our competitiveness, and economy in general down to zero. Do you want us to give you money? Let us have a decent existence, make money for our economy, because this is where the money we give you comes from.” They refuse to do so. Why? Ask them. (Knocks on the table.) That is what it is like in their heads. Those are highly incompetent people."


In [14]:
#%html
top-parts('Where the Russia-Ukraine negotiations were held?', 2, type => 'answer') 
==> data-translation(field-names => <Score Text>)
==> { $_.subst(/ ('negotia' \w+) /, { '<span style="color: orange">' ~ $0.Str ~ '</span>'}):g }()

Score,Text
0.8516614960995936,"They have been. They reached a very high stage of coordination of positions in a complex process, but still they were almost finalized. But after we withdrew our troops from Kiev, as I have already said, the other side (Ukraine) threw away all these agreements and obeyed the instructions of Western countries, European countries, and the United States to fight Russia to the bitter end. Moreover, the President of Ukraine has legislated a ban on negotiating with Russia. He signed a decree forbidding everyone to negotiate with Russia. But how are we going to negotiate if he forbade himself and everyone to do this? We know that he is putting forward some ideas about this settlement. But in order to agree on something, we need to have a dialogue. Is not that right?"
0.8359956470063451,"Why not? He considers himself head of state, he won the elections. Although we believe in Russia that the coup d’état is the primary source of power for everything that happened after 2014, and in this sense, even today’s government is flawed. But he considers himself the president, and he is recognized by the United States, all of Europe and practically the rest of the world in such a capacity – why not? He can. We negotiated with Ukraine in Istanbul, we agreed, he was aware of this. Moreover, the negotiation group leader, Mr. Arakhamia is his last name, I believe, still heads the faction of the ruling party, the party of the President in the Rada. He still heads the Presidential faction in the Rada, the country’s parliament, he still sits there. He even put his preliminary signature on the document I am telling you about. But then he publicly stated to the whole world: “We were ready to sign this document, but Mr. Johnson, then the Prime Minister of Great Britain, came and dissuaded us from doing this saying it was better to fight Russia. They would give us everything we needed to return what was lost during the clashes with Russia. And we agreed with this proposal.” Look, his statement has been published. He said this publicly. Can they return to this or not? The question is: do they want it or not? Further on, President of Ukraine issued a decree prohibiting negotiations with us. Let him cancel that decree and that’s it. We have never refused negotiations indeed. We hear all the time: is Russia ready? Yes, we have not refused! It was them who publicly refused. Well, let him cancel his decree and enter into negotiations. We have never refused. And the fact that they obeyed the demand or persuasion of Mr. Johnson, the former Prime Minister of Great Britain, seems ridiculous and very sad to me. Because, as Mr. Arakhamia put it: “We could have stopped these hostilities, this war a year and a half ago already. But the British persuaded us, and we refused this.” Where is Mr. Johnson now? And the war continues."


------

## Flavored variations

In this section we show how the spoken parts can be rephrased in the style of certain political celebrities.

Here are examples of using LLM to rephrase Tucker Carlson's questions into the style of Hillary Clinton:

In [196]:
for ^2 {
    my $q = @tcQuestions.pick;
    say '=' x 100;
    say "Tucker Carlson: $q";
    say '-' x 100;
    my $q2 = llm-synthesize(["Rephrase this question in the style of Hillary Clinton:", $q]);
    say "Hilary Clinton: $q2";
}

Tucker Carlson: And there’s a lot of that though, I think. Many nations feel upset about — there are Transylvanians as well as you, others, you know — but many nations feel frustrated by their re-drawn borders after the wars of the 20th century, and wars going back a thousand years, the ones that you mention, but the fact is that you didn’t make this case in public until two years ago in February, and in the case that you made, which I read today, you explain at great length that you thought a physical threat from the West and NATO, including potentially a nuclear threat, and that’s what got you to move. Is that a fair characterization of what you said?
----------------------------------------------------------------------------------------------------
Hilary Clinton: And there's a considerable amount of dissatisfaction in various nations, including Transylvanians like yourself, expressing their concerns about the redrawn borders after the conflicts of the 20th century and even dating 

Here are examples of using LLM to rephrase Vladimir Putin's answers into the style of Donald Trump:

In [199]:
for ^2 {
    my $q = @parts.grep({ $_.key.contains('Putin') }).pick.value;
    say '=' x 100;
    say "Vladimir Putin: $q";
    say '-' x 100;
    my $q2 = llm-synthesize(["Rephrase this question in the style of Hillary Clinton:", $q]);
    say "Donald Trump: $q2";
}

Vladimir Putin: This also confuses me. But today's German leadership is guided by the interests of the collective West rather than its national interests, otherwise it is difficult to explain the logic of their action or inaction. After all, it is not only about Nord Stream-1, which was blown up, and Nord Stream-2 was damaged, but one pipe is safe and sound, and gas can be supplied to Europe through it, but Germany does not open it. We are ready, please.
There is another route through Poland, called Yamal-Europe, which also allows for a large flow. Poland has closed it, but Poland pecks from the German hand, it receives money from pan-European funds, and Germany is the main donor to these pan-European funds. Germany feeds Poland to a certain extent. And they closed the route to Germany. Why? I don't understand.
Ukraine, to which the Germans supply weapons and give money. Germany is the second sponsor after the United States in terms of financial aid to Ukraine. There are two gas routes

-------

## References

### Articles

[AA1] Anton Antonov,
["Workflows with LLM functions"](https://rakuforprediction.wordpress.com/2023/08/01/workflows-with-llm-functions/),
(2023),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA2] Anton Antonov,
["Day 21 – Using DALL-E models in Raku"](https://raku-advent.blog/2023/12/21/day-22-using-dall-e-models-in-raku/),
(2023),
[Raku Advent Calendar blog for 2023](https://raku-advent.blog/2023).

[OAIb1] OpenAI team,
["New models and developer products announced at DevDay"](https://openai.com/blog/new-models-and-developer-products-announced-at-devday),
(2023),
[OpenAI/blog](https://openai.com/blog).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI](https://github.com/antononcube/Raku-WWW-OpenAI) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp2] Anton Antonov,
[WWW::PaLM](https://github.com/antononcube/Raku-WWW-PaLM) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp3] Anton Antonov,
[WWW::MistralAI](https://github.com/antononcube/Raku-WWW-MistralAI) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp4] Anton Antonov,
[WWW::MermaidInk](https://github.com/antononcube/Raku-WWW-MermaidInk) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp5] Anton Antonov,
[LLM::Functions](https://github.com/antononcube/Raku-LLM-Functions) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp6] Anton Antonov,
[Jupyter::Chatbook](https://github.com/antononcube/Raku-Jupyter-Chatbook) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp7] Anton Antonov,
[Image::Markup::Utilities](https://github.com/antononcube/Raku-Image-Markup-Utilities) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


### Videos

[AAv1] Anton Antonov,
["Jupyter Chatbook LLM cells demo (Raku)"](https://www.youtube.com/watch?v=cICgnzYmQZg)
(2023),
[YouTube/@AAA4Prediction](https://www.youtube.com/@AAA4prediction).

[AAv2] Anton Antonov,
["Jupyter Chatbook multi cell LLM chats teaser (Raku)"](https://www.youtube.com/watch?v=wNpIGUAwZB8),
(2023),
[YouTube/@AAA4Prediction](https://www.youtube.com/@AAA4prediction).

[AAv3] Anton Antonov
["Integrating Large Language Models with Raku"](https://www.youtube.com/watch?v=-OxKqRrQvh0),
(2023),
[YouTube/@therakuconference6823](https://www.youtube.com/@therakuconference6823).